# Tflite Model Creation for Bi-LSTM Model

TF Lite can optimize existing models to be less memory and cost-consuming, the ideal situation for using machine learning models on mobile and other Edge Devices.

In [ ]:
import numpy as  np # type: ignore
import pandas as pd # type: ignore
import tensorflow as tf # type: ignore
import math

from sklearn.preprocessing import MinMaxScaler # type: ignore
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error # type: ignore

Now,we have to convert the h5 model to tflite model 

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model('Path to the h5 model') # path to the SavedModel directory
converter.optimizations = [tf.lite.Optimize.DEFAULT] # Optimize the model  
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter._experimental_lower_tensor_list_ops = False # This is required to convert the model to tflite
tflite_model = converter.convert() # Convert the model to tflite 

with open('Path to save the tflite model', 'wb') as f: # path to save the tflite model
    f.write(tflite_model)

Now,we have to make Predictions on the Tflite model

In [ ]:
interpreter = tf.lite.Interpreter(model_path="path to the tflite model") # path to the tflite model's path
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
predicted_values = []
true_values = []

for i in range(len("x_testing_dataset")): # x_testing_dataset is the testing dataset
    cnn_input_data = np.expand_dims('x_testing_dataset'[i], axis=0).astype(np.float32) 
    interpreter.set_tensor(input_details[0]['index'], cnn_input_data) # Set the input tensor
    interpreter.invoke() # Run the model

    output_data = interpreter.get_tensor(output_details[0]['index']) # Get the output tensor

    predicted_values.append(output_data[0][0]) # Append the predicted values
    true_values.append("y_testing_dataset"[i]) # Append the true values 

predicted_values = np.array(predicted_values) 
true_values = np.array(true_values) 


predicted_values_reshaped = predicted_values.reshape(-1, 1) # Reshape the predicted values
predictions_rescaled = scaler.inverse_transform(predicted_values_reshaped).flatten() # Rescale the predicted values

y_test_reshaped = true_values.reshape(-1, 1) # Reshape the true values
y_test_rescaled = scaler.inverse_transform(y_test_reshaped).flatten()  # Rescale the true values

results = pd.DataFrame(data={'Predictions': predictions_rescaled, 'Actuals': y_test_rescaled}) # Create a dataframe with the predicted and true values
print(results)

# Error Values for the TFLite Model

In [ ]:
optimized_rmse = math.sqrt(mean_squared_error(y_test_rescaled, predictions_rescaled))
optimized_mae = mean_absolute_error(y_test_rescaled, predictions_rescaled)
optimized_mape = mean_absolute_percentage_error(predictions_rescaled, y_test_rescaled)

print("Optimized RMSE:", optimized_rmse)
print("Optimized MAE:", optimized_mae)
print("Optimized MAPE:", optimized_mape)